In [ ]:
import importlib
import numpy as np
from numba import njit
from functools import partial
import talib 
import pandas as pd
import vectorbt as vbt
import vectorbt as vbt
from vectorbt.utils.colors import adjust_opacity
from vectorbt.utils.enum_ import map_enum_fields
from vectorbt.base.reshape_fns import broadcast, flex_select_auto_nb, to_2d_array
from vectorbt.portfolio.enums import SizeType, Direction, NoOrder, OrderStatus, OrderSide
from vectorbt.portfolio import nb
import sys
import os
# function_essential_path = r"C:\Users\Jordi\Desktop\Environement de developement\Trading_Dev_Stratégie_Environement\FunctionEssential"
# indicator_liste = r'C:\Users\Jordi\Desktop\Environement de developement\Trading_Dev_Stratégie_Environement\Indicateur'
function_essential_path = r"H:\Desktop\Environement_Trading_Developement\FunctionEssential"
indicator_liste = r'H:\Desktop\Environement_Trading_Developement\Indicateur'
sys.path.append(function_essential_path)
sys.path.append(indicator_liste)
import utils as us
import s_score as score
importlib.reload(us)

In [ ]:
##FONCTION##

In [ ]:
##FONCTION LOGIQUE SIGNALS##

def entries_trades():
    pass

def exits_trades():
    pass

In [ ]:
##INDICATEUR FACTORY##

Entry = vbt.IndicatorFactory(
    class_name="entries_trades",
    input_names=[""],
    param_names=[""],
    output_names=["long","short"],
).from_apply_func(entries_trades)

Exits = vbt.IndicatorFactory(
    class_name="exits_trades",
    input_names=[""],
    param_names=[""],
    output_names=["long","short"],
).from_apply_func(exits_trades)

In [ ]:
@njit
def pre_sim_func_nb(c):
    entry_price = np.full(c.target_shape[1], np.nan, dtype=np.float64)  
    return (entry_price,)

@njit
def order_func_nb(c, entry_price, long, short, stop_loss, take_profit, size):

    price_now = nb.get_elem_nb(c, c.close)
    entry_long_now = long[c.i, c.col]
    entry_short_now = short[c.i, c.col]

    if entry_long_now:
        if c.position_now == 0:
            return nb.order_nb(
                size[c.col], 
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.001
            )
        elif c.position_now < 0:
            return nb.order_nb(
                -size[c.col],  
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.0005
            )  

    if entry_short_now:
        if c.position_now == 0:
            return nb.order_nb(
                size[c.col], 
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.001
            )
        elif c.position_now > 0:
            return nb.order_nb(
                -size[c.col],  
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.0005
            )
        

    #gestion simplifier des sorties avec TP et SL fixe 
    
    if c.position_now > 0:
        SL = entry_price[c.col] * (1 - stop_loss[c.col])
        TP = entry_price[c.col] * (1 + take_profit[c.col])
        if price_now >= TP:
            return nb.order_nb(
                -size[c.col],
                price=TP,
                direction=Direction.LongOnly,
                fees=0.0005
            )
        elif price_now <= SL:
            return nb.order_nb(
                -size[c.col],
                price=SL,
                direction=Direction.LongOnly,
                fees=0.0005
            )

    if c.position_now < 0:
        SL = entry_price[c.col] * (1 + stop_loss[c.col])
        TP = entry_price[c.col] * (1 - take_profit[c.col])
        if price_now >= SL:
            return nb.order_nb(
                -size[c.col],
                price=SL,
                direction=Direction.ShortOnly,
                fees=0.0005
            )
        elif price_now <= TP:
            return nb.order_nb(
                -size[c.col],
                price=TP,
                direction=Direction.ShortOnly,
                fees=0.0005
            )

    return NoOrder


@njit
def post_order_func_nb(c, entry_price):
    if c.order_result.status == OrderStatus.Filled:
        entry_price[c.col] = c.order_result.price

In [ ]:
##CLASS STRATEGIE##

class Strategie():

    params_fixe_un = "" #Peut etre un stop ou un TP ou autre
    params_fixe_deux = ""


    def __init__(self,data,tickers,frequence):
        self.data = data
        self.tickers = tickers
        self.frequences = frequence


    def set_params_strategie(self,params1,params2):
        self.params_fixe_un = params1
        self.params_fixe_deux = params2


    def backtest(self):
        entries_long_df = pd.DataFrame()
        entries_short_df = pd.DataFrame()
        exits_long_df = pd.DataFrame()
        exits_short_df = pd.DataFrame()
        close_df = pd.DataFrame()
        datetime_df = pd.DataFrame()

        for ticker in self.tickers:
            data = self.data[ticker]
            #Creation des entréer / Sortie / Indicateur

            trades = Entry.run(

            )

            exits = Exits.run(

                
            )

            datetime = data.reset_index()['Datetime']
            entries_long_df[ticker] = trades.long.reset_index(drop=True)
            entries_short_df[ticker] = trades.short.reset_index(drop=True)
            exits_long_df[ticker] = exit.long.reset_index(drop=True)
            exits_short_df[ticker] = exit.short.reset_index(drop=True)
            close_df[ticker] = data['close'].reset_index(drop=True)
            datetime_df[ticker] = datetime.reset_index(drop=True)

        size = np.array([1]*len(self.tickers))
        exits_long_df = exits_long_df.astype(np.float64).fillna(0)
        exits_short_df = exits_short_df.astype(np.float64).fillna(0)


        portfolio = vbt.Portfolio.from_order_func(
            close_df,  
            order_func_nb,  
            entries_long_df.values,  
            entries_short_df.values,  
            exits_long_df.values,
            exits_short_df.values,
            size,  
            freq=self.frequences,  
            init_cash=100000,  
            pre_sim_func_nb=pre_sim_func_nb,  
            post_order_func_nb=post_order_func_nb,  
            broadcast_named_args=dict(  
                long=entries_long_df.values,  
                short=entries_short_df.values,  
                exits_long_df=exits_long_df.values,  
                exits_short_df=exits_short_df.values,  
                size=size  
            )
        )
        us.rapport_backtest(portfolio,close_df,datetime_df,self.tickers)
        return portfolio

    def optimize(self,fitness_func,choice,period1,period2,params1,params2):
         
         if choice == 1:

            combinaison=[]
            for i in params1:
                for j in params2:
                    combinaison.append((i,j))
                    #Peut rajouter un troisieme param pour faire l'opti de 3 params (grand max)

            
            entries = Entry.run(param_product=True)


            long = entries.entries_long
            short = entries.entries_short

            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            close = close.reset_index(drop=True)
            close = pd.DataFrame(close)  
            close = close.iloc[:, :long.shape[1]]

            capital = 100000  
            size = np.array([(0.05 * capital) / (0.02 * close.iloc[0, i]) for i in range(close.shape[1])])
            size = np.array([size[0]] * long.shape[1])
            stop_loss = np.array([0.02] * long.shape[1])  
            take_profit = np.array([0.03] * long.shape[1])
            
            portfolio = vbt.Portfolio.from_order_func(
                close,  
                order_func_nb,
                long.values,  
                short.values,  
                stop_loss,  
                take_profit,  
                size,  
                freq=self.frequence_exec,
                init_cash=100000,
                pre_sim_func_nb=pre_sim_func_nb,
                post_order_func_nb=post_order_func_nb,
                broadcast_named_args=dict(
                    long=long.values,
                    short=short.values,
                    stop_loss=stop_loss,  
                    take_profit=take_profit,
                    size=size 
                )
            )

            perf = us.get_heatmap(portfolio,combinaison,fitness_func)
            param1,param2,metrics = us.get_best_param(perf)
            return param1,param2


    def wfa(self,period_OOS, period_IOS,params1):
 
        close_df = pd.DataFrame()

        for ticker in self.tickers:

            data = self.data[ticker]
            cycle = us.generate_wf_cycles(data, period_IOS, period_OOS)

            close = data['close']
            high = data['high']
            low = data['low']

            long_entries_index = []
            short_entries_index = []
            close_df[ticker] = close

            for i in range(len(cycle)):

                close_IOS = data["close"].iloc[cycle[i][0]:cycle[i][1]]
                close_OOS = data["close"].iloc[cycle[i][2]:cycle[i][3]]

                high_IOS = data["high"].iloc[cycle[i][0]:cycle[i][1]]
                high_OOS = data["high"].iloc[cycle[i][2]:cycle[i][3]]

                low_IOS = data["low"].iloc[cycle[i][0]:cycle[i][1]]
                low_OOS = data["low"].iloc[cycle[i][2]:cycle[i][3]]




                param1, param2 = self.optimize()

                #premiere itération on save la période IOS pour afficher l'entierté de mon jeu de donnée 
                if i == 0:
                    entries = Entry.run()
                    long_entries_index.extend(entries.entries_long[entries.entries_long].index)
                    short_entries_index.extend(entries.entries_short[entries.entries_short].index)


                entries = Entry.run()
                long_entries_index.extend(entries.entries_long[entries.entries_long].index)
                short_entries_index.extend(entries.entries_short[entries.entries_short].index)

            #Je copie close je remplace tout par False et je remplace par True les indices donnée par Entry
            entries_long = data["close"].copy()
            entries_long[:] = False 
            entries_long.loc[long_entries_index] = True  
            entries_short = data["close"].copy()
            entries_short[:] = False  
            entries_short.loc[short_entries_index] = True  
            #Mettre entries long et short dans le bon format 
            entries_long = entries_long.to_frame() 
            entries_short = entries_short.to_frame() 
            entries_long = entries_long.reset_index(drop=True) 
            entries_short = entries_short.reset_index(drop=True)  
            entries_long.columns = [ticker] 
            entries_short.columns = [ticker]
            entries_long = entries_long.astype(bool)  
            entries_short = entries_short.astype(bool)



            capital = 100000   
            size = np.array([(0.05 * capital) / (0.02 * close_df.iloc[0, i]) for i in range(len(self.tickers))])
            stop_loss = np.array([0.01] * len(self.tickers))  
            take_profit = np.array([0.02] * len(self.tickers))

        
            #Si ca fonctionne pas regarde bien que les shapes soit cohérente, le type de entries long et short ca doit etre des boolean

            portfolio = vbt.Portfolio.from_order_func(
                close_df,  
                order_func_nb,  
                entries_long.values,  
                entries_short.values,  
                stop_loss,  
                take_profit,  
                size,  
                freq=self.frequence_exec,  
                init_cash=100000,  
                pre_sim_func_nb=pre_sim_func_nb,  
                post_order_func_nb=post_order_func_nb,  
                broadcast_named_args=dict(  
                    long=entries_long.values,  
                    short=entries_short.values,  
                    stop_loss=stop_loss,  
                    take_profit=take_profit,  
                    size=size  
                )
            )
            us.get_pnl(portfolio)
            return portfolio



In [ ]:
##UTILISATION DE LA STRAT##
tickers = ['EURUSD','GPDUSD']
DATA = us.get_data_forex(tickers,'H4')
strat = Strategie(DATA,tickers,"240m","240m",0)